# Tube Learning Notebook

## DataFrame Construction

In [1]:
import pandas as pd
import ast
import numpy as np
import glob

In [2]:
# Function to safely evaluate lists
def safe_eval(col):
    try:
        return ast.literal_eval(col)
    except ValueError:
        return col  # Return as is if it's not a string representation of a list

# Initialize an empty DataFrame
all_data = pd.DataFrame()

# Set the number of robots you want to include
num_robots = 5  # Set the number of robots to include
robot_indices = list(range(num_robots))  # Generates a list [0, 1, 2, ..., num_robots-1]

# Use glob to find all the files that match the pattern
file_list = glob.glob('data/trajectory_data_*.csv')

# Loop through the files sorted to maintain the order
for filename in sorted(file_list):
    temp_df = pd.read_csv(filename)
    # Filter the DataFrame to only include rows where the robot_index is in the list of desired indices
    temp_df = temp_df[temp_df['robot_index'].isin(robot_indices)]
    # Apply transformations right after reading
    temp_df['joint_positions'] = temp_df['joint_positions'].apply(safe_eval)
    temp_df['joint_velocities'] = temp_df['joint_velocities'].apply(safe_eval)
    all_data = pd.concat([all_data, temp_df], ignore_index=True)


In [3]:
# Now create the derived columns
all_data['x_t'] = all_data.apply(lambda row: row['joint_positions'] + row['joint_velocities'], axis=1)
all_data['u_t'] = all_data.apply(lambda row: [row['velocity_x'], row['velocity_y']], axis=1)
all_data['z_t'] = all_data.apply(lambda row: [row['traj_x'], row['traj_y']], axis=1)
all_data['v_t'] = all_data.apply(lambda row: [row['reduced_command_x'], row['reduced_command_y']], axis=1)

# Calculate the vector differences without norm for w_xy_t and shift it for w_xy_{t+1}
all_data['w_xy_t'] = all_data.apply(lambda row: [row['position_x'] - row['traj_x'], row['position_y'] - row['traj_y']], axis=1)
all_data['group'] = all_data['episode_number'].astype(str) + '_' + all_data['robot_index'].astype(str)

# Original calculation for w_t for reference
all_data['w_t'] = np.sqrt((all_data['position_x'] - all_data['traj_x'])**2 + (all_data['position_y'] - all_data['traj_y'])**2)

# Shift operations for next timestep values
all_data['x_{t+1}'] = all_data.groupby('group')['x_t'].shift(-1)
all_data['z_{t+1}'] = all_data.groupby('group')['z_t'].shift(-1)
all_data['w_{t+1}'] = all_data.groupby('group')['w_t'].shift(-1)
all_data['w_xy_{t+1}'] = all_data.groupby('group')['w_xy_t'].shift(-1)

# Function to drop the first and last 10 data points from each episode
def drop_edges(group):
    return group.iloc[10:-10]
all_data = all_data.groupby('group', group_keys=False).apply(drop_edges)

# Drop rows where x_{t+1}, z_{t+1}, w_{t+1}, and w_xy_{t+1} do not exist
all_data.dropna(subset=['x_{t+1}', 'z_{t+1}', 'w_{t+1}', 'w_xy_{t+1}'], inplace=True)

# Select and order the final columns
final_df = all_data[['group', 'x_t', 'u_t', 'z_t', 'v_t', 'w_t', 'w_xy_t', 'x_{t+1}', 'z_{t+1}', 'w_{t+1}', 'w_xy_{t+1}']]


We have $D=\{\omega_t, x_t, u_t, z_t, v_t, \omega_{t+1}, x_{t+1}, z_{t+1}\}$:

In [4]:
# Print the final DataFrame
final_df.head()

,group,x_t,u_t,z_t,v_t,w_t,w_xy_t,x_{t+1},z_{t+1},w_{t+1},w_xy_{t+1}
50,1.0_0,"[0.021700723096728325, 0.5511646866798401, -0....","[0.5442334846271863, -0.283746258114005]","[0.0839964397055372, -0.0443801499923683]","[0.4199822079150199, -0.2219007549217102]",0.026090,"[-0.0259041007415884, 0.0031106175297319003]","[0.023384274914860725, 0.5582820773124695, -0....","[0.0923960836760909, -0.0488181649916052]",0.029730,"[-0.029268851125635, 0.005218078390838196]"
55,1.0_0,"[0.023384274914860725, 0.5582820773124695, -0....","[0.5480042460848393, -0.2824143952132594]","[0.0923960836760909, -0.0488181649916052]","[0.4199822079150199, -0.2219007549217102]",0.029730,"[-0.029268851125635, 0.005218078390838196]","[0.02588886395096779, 0.5638883709907532, -0.7...","[0.1007957276466446, -0.053256179990842]",0.033340,"[-0.032551680733351795, 0.0072078736123690965]"
60,1.0_0,"[0.02588886395096779, 0.5638883709907532, -0.7...","[0.550880471042393, -0.2824574469904211]","[0.1007957276466446, -0.053256179990842]","[0.4199822079150199, -0.2219007549217102]",0.033340,"[-0.032551680733351795, 0.0072078736123690965]","[0.028353633359074593, 0.5677175521850586, -0....","[0.1091953716171983, -0.0576941949900788]",0.036844,"[-0.0357105226042108, 0.0090702395005937]"
65,1.0_0,"[0.028353633359074593, 0.5677175521850586, -0....","[0.5525190403579519, -0.282065262418896]","[0.1091953716171983, -0.0576941949900788]","[0.4199822079150199, -0.2219007549217102]",0.036844,"[-0.0357105226042108, 0.0090702395005937]","[0.030368400737643242, 0.5698843002319336, -0....","[0.1175950155877521, -0.0621322099893157]",0.040191,"[-0.0387335787472045, 0.0107260937201605]"
70,1.0_0,"[0.030368400737643242, 0.5698843002319336, -0....","[0.5536589812999309, -0.2793862519163999]","[0.1175950155877521, -0.0621322099893157]","[0.4199822079150199, -0.2219007549217102]",0.040191,"[-0.0387335787472045, 0.0107260937201605]","[0.031975340098142624, 0.5703613758087158, -0....","[0.1259946595583058, -0.0665702249885525]",0.043377,"[-0.041641720871798904, 0.012145407415824198]"


Optional Saving:

In [20]:
final_df.to_csv('processed_trajectory_data.csv', index=False)

## Network Construction

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import ast
from tqdm import tqdm
import wandb

wandb.login(key="70954bb73c536b7f5b23ef315c7c19b511e8a406")

wandb: Currently logged in as: coleonguard (coleonguard-Georgia Institute of Technology). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/colejohnson/.netrc


True

Load in the data:

In [2]:
def safe_eval(col):
    try:
        return ast.literal_eval(col)
    except ValueError:
        return col  # Return as is if it's not a string representation of a list

def convert_to_tensor_input(row):
    flat_list = []
    for item in row:
        if isinstance(item, list):
            flat_list.extend(item)
        else:
            flat_list.append(item)
    return flat_list

def load_and_prepare_data(filename, tube_type):
    df = pd.read_csv(filename)
    list_columns = ['x_t', 'u_t', 'z_t', 'v_t'] + (['w_xy_t', 'w_xy_{t+1}'] if tube_type == 'rectangular' else [])
    feature_columns = ['x_t', 'u_t', 'z_t', 'v_t']
    
    for col in list_columns:
        df[col] = df[col].apply(safe_eval)  # Assumes safe_eval correctly parses strings into lists

    X = torch.tensor(df[feature_columns].apply(convert_to_tensor_input, axis=1).tolist(), dtype=torch.float32)
    
    if tube_type == 'rectangular':
        target_columns = ['w_xy_t', 'w_xy_{t+1}']
    else:
        target_columns = ['w_t', 'w_{t+1}']

    # Construct the target tensor manually from rows
    y_data = df[target_columns].apply(lambda row: [row[col] for col in target_columns], axis=1).tolist()
    y = torch.tensor(y_data, dtype=torch.float32)
    
    return X, y

def create_data_loaders(X, y, batch_size=64):
    # Create a TensorDataset
    dataset = TensorDataset(X, y[:, 1].unsqueeze(1))  # Assumes the target is at the second position

    # Split data into train and test sets
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    # Create DataLoader objects
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    return train_loader, test_loader

Model specifications:

In [3]:
class TubeWidthPredictor(nn.Module):
    def __init__(self, input_size, num_units, num_layers, output_dim=1):
        super(TubeWidthPredictor, self).__init__()
        self.layers = nn.ModuleList([nn.Linear(input_size, num_units), nn.ReLU()])
        for _ in range(num_layers - 1):
            self.layers.append(nn.Linear(num_units, num_units))
            self.layers.append(nn.ReLU())
        self.layers.append(nn.Linear(num_units, output_dim))  # Dynamically set output dimension

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

class AsymmetricLoss(nn.Module):
    def __init__(self, alpha=0.9, delta=1.0):
        super(AsymmetricLoss, self).__init__()
        self.alpha = alpha
        self.huber = nn.HuberLoss(delta=delta)

    def forward(self, y_pred, y_true, tube_type):
        if tube_type == 'sphere':
            residual = y_true - y_pred
            loss = torch.where(residual <= 0, self.alpha * residual, (1 - self.alpha) * residual.abs())
            return self.huber(loss, torch.zeros_like(loss))
        elif tube_type == 'rectangular':
            # Compute L2 norm of the residuals for each component
            y_true = y_true.squeeze(1) # because im a lazy idiot
            residual_x = y_true[:, 0] - y_pred[:, 0]
            residual_y = y_true[:, 1] - y_pred[:, 1]
            norm_residual = torch.sqrt(residual_x**2 + residual_y**2)
            loss = torch.where(norm_residual <= 0, self.alpha * norm_residual, (1 - self.alpha) * norm_residual.abs())
            return self.huber(loss, torch.zeros_like(loss))

# Modify train_and_test function
def train_and_test(model, criterion, optimizer, train_loader, test_loader, tube_type, num_epochs=500):
    for epoch in tqdm(range(num_epochs), desc="Epochs", position=0, leave=True):
        model.train()
        train_loss = 0
        for data, targets in tqdm(train_loader, desc="Training Batches", leave=False):
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, targets, tube_type)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Modify evaluation based on tube_type
        test_loss, metrics = evaluate_model(model, test_loader, criterion, tube_type)
        wandb.log({'Train Loss': train_loss, 'Test Loss': test_loss, **metrics, 'Epoch': epoch})

def evaluate_model(model, test_loader, criterion, tube_type):
    model.eval()
    test_loss = 0
    differences = []
    total_predictions = 0  # Total number of predictions
    count_y_pred_gt_wt1 = 0  # Count of predictions greater than target

    with torch.no_grad():
        for data, targets in test_loader:
            outputs = model(data)
            test_loss += criterion(outputs, targets, tube_type).item()

            # Increase total predictions
            total_predictions += outputs.size(0)

            if tube_type == 'sphere':
                # Sphere-specific metric calculations
                greater_mask = outputs > targets
                count_y_pred_gt_wt1 += greater_mask.sum().item()
                # Calculate differences where predictions are less than targets
                less_mask = outputs < targets
                differences.extend((targets[less_mask] - outputs[less_mask]).abs().tolist())

            elif tube_type == 'rectangular':
                # Rectangular-specific metric calculations
                targets = targets.squeeze(1) # again... singular braincell human being...
                outputs_x_norm = outputs[:, 0]
                outputs_y_norm = outputs[:, 1]
                targets_x_norm = targets[:, 0]
                targets_y_norm = targets[:, 1]
                
                # Check if either x or y prediction is greater than the target
                greater_mask_x = outputs_x_norm > targets_x_norm
                greater_mask_y = outputs_y_norm > targets_y_norm
                count_y_pred_gt_wt1 += (greater_mask_x | greater_mask_y).sum().item()

                # Calculate differences for x and y separately where predictions are less than targets
                differences_x = (targets_x_norm - outputs_x_norm)[outputs_x_norm < targets_x_norm].abs().tolist()
                differences_y = (targets_y_norm - outputs_y_norm)[outputs_y_norm < targets_y_norm].abs().tolist()
                differences.extend(differences_x)
                differences.extend(differences_y)

    avg_diff = sum(differences) / len(differences) if differences else 0
    test_loss /= len(test_loader)
    proportion_y_pred_gt_wt1 = count_y_pred_gt_wt1 / total_predictions if total_predictions > 0 else 0
    
    metrics = {
        'Test Loss': test_loss,
        'Proportion y_pred > w_{t+1}': proportion_y_pred_gt_wt1,
        'Avg Abs Diff y_pred < w_{t+1}': avg_diff
    }
    return test_loss, metrics

In [4]:
def main():
    wandb.init()
    config = wandb.config
    tube_type = config.tube_type
    filename = 'processed_trajectory_data.csv'
    
    X, y = load_and_prepare_data(filename, tube_type)
    train_loader, test_loader = create_data_loaders(X, y, batch_size=64)

    # Set input size based on tube type
    input_size = 30
    output_dim = 2 if tube_type == 'rectangular' else 1  # Set output dimension based on tube type

    model = TubeWidthPredictor(input_size=input_size, num_units=config.num_units, num_layers=config.num_layers, output_dim=output_dim)
    criterion = AsymmetricLoss(alpha=config.alpha, delta=1.0)
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
    
    train_and_test(model, criterion, optimizer, train_loader, test_loader, tube_type, num_epochs=config.num_epochs)

    wandb.finish()

In [ ]:
# Hyperparameter sweep and project setup
alpha_values = [0.8, 0.9, 0.95, 0.99, 0.999]
tube_types = ['rectangular', 'sphere']

for alpha in alpha_values:
    for tube_type in tube_types:
        sweep_config = {
            'method': 'grid',
            'metric': {'name': 'Test Loss', 'goal': 'minimize'},
            'parameters': {
                'alpha': {'value': alpha},
                'learning_rate': {'values': [0.001]},
                'num_units': {'values': [32]},
                'num_layers': {'values': [2]},
                'tube_type': {'value': tube_type},
                'num_epochs': {'value': 500}  # Example of defining epoch in config
            }
        }
        project_name = f"tube_width_experiment_alpha_{alpha}_{tube_type}"
        sweep_id = wandb.sweep(sweep_config, project=project_name)
        wandb.agent(sweep_id, main)

Create sweep with ID: y4nkty3n
Sweep URL: https://wandb.ai/coleonguard-Georgia%20Institute%20of%20Technology/tube_width_experiment_alpha_0.8_rectangular/sweeps/y4nkty3n


wandb: Agent Starting Run: zkuyhd2o with config:
wandb: 	alpha: 0.8
wandb: 	learning_rate: 0.001
wandb: 	num_epochs: 500
wandb: 	num_layers: 2
wandb: 	num_units: 32
wandb: 	tube_type: rectangular


Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|     

Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|     

Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|     

...

Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|     

Epochs:  32%|██████████▋                      | 161/500 [00:22<00:58,  5.80it/s]


Epochs:  39%|████████████▊                    | 194/500 [00:27<00:38,  7.88it/s]


Training Batches:  79%|██████████████████▉     | 49/62 [00:00<00:00, 481.86it/s]


Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]
                                                                                
Training Batches:   0%|                                  | 0/62 [00:00<?, ?it/s]